In [10]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Kamja, I live in South Korea", "Wow that is so cool!")

In [11]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [12]:
get_history()

{'history': 'Kamja from South Korea introduces themselves to the AI, who responds with enthusiasm, finding it cool. The human mentions how pretty South Korea is, and the AI expresses a wish to visit.'}